In [ ]:
import sqlite3
# Connect to database
conn = sqlite3.connect('pincode.db')
cursor = conn.cursor()

#Create table
query = '''
CREATE TABLE Postal_Offices (
    CircleName VARCHAR(255),
    RegionName VARCHAR(255),
    DivisionName VARCHAR(255),
    OfficeName VARCHAR(255),
    Pincode INTEGER,
    OfficeType VARCHAR(255),
    Delivery VARCHAR(255),
    District VARCHAR(255),
    StateName VARCHAR(255)
);

'''
cursor.execute(query)
#Read csv
import pandas as pd
df = pd.read_csv('Pincode_30052019.csv',encoding='ISO-8859-1')
df.columns =['CircleName', 'RegionName', 'DivisionName', 'OfficeName', 'Pincode',
       'OfficeType', 'Delivery', 'District', 'StateName']

#Import the csv into database
df.to_sql('Postal_Offices', conn, if_exists='append', index=False)
conn.close()

OperationalError: table Postal_Offices already exists

In [ ]:
!pip install langchain

In [ ]:
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 71.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 150.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 212.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 232.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.55-cp310-cp310-manylinux_2_35_x86_64.whl size=2785231 sha256=bfae5bf1468af56262bbd614314378defbba1aa5ab2b7d8f2c813a869b528ff8
  Stored in directory: /tmp/pip-ephem-wheel-cache-m59wj90u/wheels/35/c9/bb/019dbfeef119ab5c29f76574b76070afa7b7755ccfbb3ee226
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.

In [ ]:
#!wget 'https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf'

--2024-03-05 06:12:38--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.23, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/72/62/726219e98582d16c24a66629a4dec1b0761b91c918e15dea2625b4293c134a92/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.2.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.2.Q4_K_M.gguf%22%3B&Expires=1709878358&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTg3ODM1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzcyLzYyLzcyNjIxOWU5ODU4MmQxNmMyNGE2NjYyOWE0ZGVjMWIwNzYxYjkxYzkxOGUxNWRlYTI2MjViNDI5M2MxMzRhOT

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
n_gpu_layers = 32  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase.from_uri("sqlite:///pincode.db")
llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    n_ctx=4096, # Context window
    temperature = 0.4,
)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "What is address of pincode 800020"})
print(response)

 SELECT OfficeName FROM "Postal_Offices" WHERE Pincode = 800020;


llama_print_timings:        load time =  240555.32 ms
llama_print_timings:      sample time =      16.00 ms /    28 runs   (    0.57 ms per token,  1750.44 tokens per second)
llama_print_timings: prompt eval time =  286555.80 ms /   608 tokens (  471.31 ms per token,     2.12 tokens per second)
llama_print_timings:        eval time =   17914.67 ms /    27 runs   (  663.51 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =  304637.42 ms /   635 tokens


SELECT OfficeName FROM "Postal_Offices" WHERE Pincode = 800020;


In [ ]:
cursor.execute(response)
print(cursor.fetchall())

[('Ashok Nagar SO Patna',), ('Chitragupta Nagar SO',), ('DHELMA BO',), ('KSector SO',), ('LOHIA NAGAR S.O',), ('RMS Colony SO',), ('West Lohianagar SO',)]
